In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from tqdm import tqdm
import numpy as np
import json
from gensim.models.fasttext import FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

# Данные
**коса** - прическа и рельеф.

### рельеф

* Основной и газетный корпуса НКРЯ
* Несколько контекстов
  * песчаный|залив|отлог|вода|море в контексте от -15 до +15
  * *на* перед *коса*
* Удалить примеры с «Тузла», «Куршская»
* чистка мусора

### прическа

* Основной корпус НКРЯ
* заплести|расплести|сплести|плести|распустить|волосы|девушка|женщина|девочка|голова|отрезать в контексте от -15 до +15
* удалить все примеры с «пробор» (там были нормальные, но у нас и без них много примеров)

еще удалить дубликаты из примеров.

Итого:
* 571 коса-рельеф (0)
* 589 коса-прическа (1)

Всего - 1160 примеров. <br>
Вообще, кажется, ситуация довольно искусственная, потому что самыми важными признаками для классификатора скорее всего станут те контекстные слова, по которым я вытаскивала примеры. Но мне все же кажется, что более-менее это отражает некоторую реальность, хотя честная ручная разметка всех примеров подряд была бы лучше, а в реальных проектах уж точно. Зато можно порадоваться высокому качеству (spoiler) хоть где-то :)

# Препроцессинг

* токенизатор+лемматизатор – mystem. Проблема pymorphy2 – лемматизирует «косу» как «кос», а «косы» как «коса».
* Стоп-слова - из NLTK минус "на" (важный признак для косы-рельефа) и плюс "коса", чтобы удобно ее выкинуть и учитывать только контекст. Стоп-слова выкидываю на стадии векторизации.

In [3]:
stop = set(stopwords.words('russian')) - set(['на'])
stop.add('коса')
m = Mystem(entire_input=False)

In [16]:
def get_data(filename):
    full_examples = []
    data = []
    target = []
    with open(filename,'r',encoding='utf-8') as f:
        for line in tqdm(f.readlines()):
            t,d = line.strip().split('\t')
            full_examples.append(d.strip())
            d_lemmas = m.lemmatize(d)
            data.append(' '.join(d_lemmas))
            target.append(int(t))
    return full_examples,data,target

In [17]:
full_examples,data,target = get_data('examples.txt')

100%|██████████████████████████████████████| 1160/1160 [19:19<00:00,  1.00it/s]


In [18]:
# записать лемматизацию (mystem немного долгий) и остальное заодно, чтобы не парсить сырой файл потом
with open('examples_lemmatized.json','w',encoding='utf-8') as f:
    s = (json.dumps({'full_examples':full_examples,'data':data,'target':target},ensure_ascii=False))
    f.write(s)

In [4]:
# открыть лемматизацию
with open('examples_lemmatized.json','r',encoding='utf-8') as f:
    d = json.load(f)
data = d['data']
target = d['target']
full_examples = d['full_examples']

# Векторизация
* TfidfVectorizer
* Средние векторов из дистрибутивной семантики (FastText)

In [5]:
tfidf = TfidfVectorizer(stop_words=stop)
data_tfidf = tfidf.fit_transform(data)

In [6]:
def distrib_vect(sentence):
    tokens = sentence.split(' ')
    vects = []
    for token in tokens: # add pmi weight?
        if token not in stop and token in distrib:
            vects.append(distrib[token])
    return np.average(np.array(vects),axis=0)

In [7]:
distrib = FastText.load('m/araneum_none_fasttextskipgram_300_5_2018.model')

In [8]:
data_distrib = [distrib_vect(x) for x in data]

C:\Users\Андрей\Anaconda3\envs\python36\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Андрей\Anaconda3\envs\python36\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# Классификация
На дефолтных параметрах:
* Logistic Regression
* Gradient Boosting Classifier
* Random Forest Classifier

In [9]:
def get_preds(data,target,clf):
    y_pred = cross_val_predict(clf,data,target,cv=5)
    print(f1_score(target,y_pred))
    return y_pred

In [10]:
data_dict = {
    'TfidfVectorizer': data_tfidf,
    'FastText': data_distrib
}

clfs = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=42),
    'Random Forest Classifier': RandomForestClassifier(random_state=42)
}

In [11]:
for clf_label,clf in clfs.items():
    for d_label,d in data_dict.items():
        print(clf_label,'&',d_label)
        _ = get_preds(d,target,clf)
    print()

Logistic Regression & TfidfVectorizer
0.9881355932203391
Logistic Regression & FastText
0.9727891156462585

Gradient Boosting Classifier & TfidfVectorizer
0.9837745516652434
Gradient Boosting Classifier & FastText
0.9677966101694915

Random Forest Classifier & TfidfVectorizer
0.9522983521248916
Random Forest Classifier & FastText
0.9343696027633851



Самая лучшая комбинация - Logistic Regression & TfidfVectorizer. FastText везде сработал хуже. Возможно, к нему лучше добавить веса PMI слов из контекста и рассматриваемого слова.<br>
Дальше будем рассматривать только Logistic Regression & TfidfVectorizer.

# Ошибки

In [12]:
clf = LogisticRegression(random_state=42)
y_pred = get_preds(data_tfidf,target,clf)

0.9881355932203391


In [13]:
error_idx = np.where(y_pred != np.array(target))
print('Total',len(error_idx[0]),'errors')
print('true','pred','sentence')
for e in error_idx[0]:
    print(target[e],y_pred[e],full_examples[e])

Total 14 errors
true pred sentence
0 1 ― Гээнта спит где-нибудь на косе, ― сказал свое Кялундзига.
0 1 Ксюта подняла голову из воды и увидела на галечной косе рядом со своей одеждой чернобородого человека недеревенской наружности в клетчатой ковбойке с закатанными рукавами, отмахивавшегося геологическим молотком от разъяренно прыгающего вокруг него Чарли.
0 1 Наталка шептала без умолку обо всем, что ей приходило в голову, ― обо всех новостях на косе, о том, например, что, говорят, в степи ходит по шляхам старуха с железными глазами и на кого ни глянет ― у того непременно убьют кого-нибудь на войне.
0 1 Белые медведи знают это и в голодные весенние месяцы выходят на косу.
0 1 Он поднял вдоль статуи голову и увидел, что Царь ни черта не видит в той стороне, в которую плыла девочка: ни Волги, ни Бирючьей косы, ни моря, ни персидских сладких, как клубника, лимонов, ни жарких берегов, где девочка могла бы хорошенько погреться…
0 1 Она приняла руку, хотя чувствовала себя вполне уверенно.   И

Всего 14 ошибок на всем корпусе.

Во многих случаях, где в действительности коса-рельеф, а классификатор считает, что это коса-прическа, это "на косе" и отсутствие других ключевых для косы-рельефа слов.

Там, где ошибка в сторону косы-рельефа для контекстов косы-прически, фигурируют ключевые слова для косы-рельефа (вода, волны), причем первая ошибка - это мусор, коса-инструмент, и последняя - тоже неверный класс в разметке (из-за слова "девушка").

Возможно, что такая зависимость от ключевых слов лечится честной разметкой :)

# Уменьшение размерности
PCA - до 5 измерений (уменьшить примерно в 1000 раз).


In [14]:
pca = PCA(n_components = 5, random_state = 42)
data_tfidf_pca = pca.fit_transform(data_tfidf.todense())
_ = get_preds(data_tfidf_pca,target,clf)

0.9796954314720812


Качество ухудшилось, но ненамного для такого уменьшения размерности.
# Другие примеры
Взяты из Araneum Russicum Minus по запросу "коса" (без контекстных слов).

In [17]:
test_examples,test_data,test_target = get_data('test_examples.txt')
test_tfidf_data = tfidf.transform(test_data)

100%|██████████████████████████████████████████| 10/10 [00:09<00:00,  1.01it/s]


In [18]:
clf.fit(data_tfidf,target) # обучаем на всем
y_pred = clf.predict(test_tfidf_data)

print('true','pred','sentence')
for i,pred in enumerate(y_pred):
    print(test_target[i],pred,test_examples[i]) 

true pred sentence
1 1 Для женщины однако, во все времена и страны, длинные косы считались украшением.
1 1 Невеста заплетала шесть кос и, перевязав их красной лентой, укладывала вокруг головы.
1 1 А XIX век подарил дамам прическу «бараний рог»: косы плели не с самого начала локона, а с средины.
1 1 Мама сегодня в цветастом крепдешиновом платье, ее большая, золотистая коса уложена венком по тогдашней моде, но особенно красив отец в летной фуражке, зеленой гимнастерке с полыхающими на солнце погонами!
1 1 Вынуждена была отрезать косу , потому, что одной рукой не могла ее расчёсывать.
0 0 Самой интересной особенностью Азовского моря является наличие на его побережье большого количества кос.
0 0 Но упёрлись в песчаную косу, далёко уходящую в море.
0 0 Генеральным планом развития Евпатории определено новое место под порт – район Южной косы озера Донузлав, где расположен грузовой район по добыче и транспортировке песка.
0 0 Почти прямо напротив площадки (чуть правее) в море выдавалась своеоб

Все предсказания верны.